In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xvf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.5.1-bin-hadoop3.tgz'


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from google.colab import files
uploaded = files.upload()

Saving testItem.data.zip to testItem.data.zip
Saving trainItem.data.zip to trainItem.data.zip


In [7]:
uploaded = files.upload()

Saving re_u.data to re_u.data


In [8]:
!unzip testItem.data.zip
!unzip trainItem.data.zip

Archive:  testItem.data.zip
  inflating: testItem.data           
Archive:  trainItem.data.zip
  inflating: trainItem.data          


## Part 1

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [10]:
training = spark.read.csv("trainItem.data", header = False)
training.show(5)

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|199808|248969| 90|
|199808|  2663| 90|
|199808| 28341| 90|
|199808| 42563| 90|
|199808| 59092| 90|
+------+------+---+
only showing top 5 rows



In [11]:
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
training.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|    90|
|199808|  2663|    90|
|199808| 28341|    90|
|199808| 42563|    90|
|199808| 59092|    90|
+------+------+------+
only showing top 5 rows



In [12]:
from pyspark.sql.types import IntegerType
training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))
training.show(3)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
+------+------+------+
only showing top 3 rows



In [13]:
als = ALS(
    maxIter=30,
    rank = 20,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="nan"
)

In [17]:
model = als.fit(training)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
testing = spark.read.csv("testItem.data", header = False)

In [ ]:
testing = testing.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
testing.show(5)

In [16]:
testing = testing.withColumn("userID", testing["userID"].cast(IntegerType()))
testing = testing.withColumn("itemID", testing["itemID"].cast(IntegerType()))
testing = testing.withColumn("rating", testing["rating"].cast('float'))
testing.show(3)

NameError: name 'testing' is not defined

In [ ]:
predictions = model.transform(testing)
predictions.show(5)

In [ ]:
predictions_df = predictions.toPandas()

In [ ]:
predictions_df['prediction'] = predictions_df['prediction'].fillna(predictions_df['prediction'].mean())

In [ ]:
predictions_df['TrackID'] = predictions_df['userID'].astype(str) + '_' + predictions_df['itemID'].astype(str)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.concat([
    predictions_df.sort_values('prediction').groupby(['userID']).head(3).assign(Predictor=0),
    predictions_df.sort_values('prediction').groupby(['userID']).tail(3).assign(Predictor=1),
])[['TrackID', 'Predictor']].to_csv('predict.csv', index=None)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
data = spark.read.csv('re_u.data', header=False).withColumnRenamed('_c0', 'userID').withColumnRenamed('_c1', 'itemID').withColumnRenamed('_c2', 'rating')
data = data.withColumn("userID", data["userID"].cast(IntegerType()))
data = data.withColumn("itemID", data["itemID"].cast(IntegerType()))
data = data.withColumn("rating", data["rating"].cast('float'))

In [ ]:
scores = []

In [ ]:
for rank in [5, 7, 10, 20]:
    als = ALS(
        maxIter = 20,
        rank = rank,
        userCol="userID",
        itemCol="itemID",
        ratingCol="rating",
        nonnegative = True,
        implicitPrefs = False,
        coldStartStrategy="nan"
    )
    model = als.fit(data)
    prediction = model.transform(data)
    mse = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='mse').evaluate(prediction)
    scores.append({'maxIter': 20, 'rank': rank, 'mse': mse, 'scenario': 1})

In [ ]:
for maxIter in [2, 5, 10, 20]:
    als = ALS(
        maxIter = maxIter,
        rank = 20,
        userCol="userID",
        itemCol="itemID",
        ratingCol="rating",
        nonnegative = True,
        implicitPrefs = False,
        coldStartStrategy="nan"
    )
    model = als.fit(data)
    prediction = model.transform(data)
    mse = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='mse').evaluate(prediction)
    scores.append({'maxIter': maxIter, 'rank': 20, 'mse': mse, 'scenario': 2})

In [ ]:
for size in [2000, 5000, 10000, 20000, 50000, 100000]:
    data_sample = data.limit(size)
    als = ALS(
        maxIter = 20,
        rank = 20,
        userCol="userID",
        itemCol="itemID",
        ratingCol="rating",
        nonnegative = True,
        implicitPrefs = False,
        coldStartStrategy="nan"
    )
    model = als.fit(data_sample)
    prediction = model.transform(data).na.drop(how="any")
    mse = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='mse').evaluate(prediction)
    scores.append({'maxIter': 20, 'rank': 20, 'size': size, 'mse': mse, 'scenario': 3})

In [ ]:
pd.DataFrame(scores)#.sort_values('mse')